Instalacja potrzebnych modulow

In [1]:
!pip install opencv-python pytesseract
!sudo apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (11.0 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [2]:
!sudo apt-get install -y tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


**Wersja 1 - przed poprawką.**

In [4]:
import cv2
import pytesseract
import os

def process_image(image_path, debug=False):
    # Tworzenie folderu debug
    debug_folder = "debug"
    if debug:
        os.makedirs(debug_folder, exist_ok=True)

    # Wczytanie obrazu
    image = cv2.imread(image_path)

    # Konwersja do skali szarości
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "gray.jpg"), gray)

    # Redukcja szumów
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "blurred.jpg"), blurred)

    # Adaptacyjne progowanie
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "thresh.jpg"), thresh)

    # Znalezienie konturów
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    license_plate = None
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h
        if 2 < aspect_ratio < 5 and h > 20:
            license_plate = gray[y:y+h, x:x+w]
            if debug:
                plate_debug = image[y:y+h, x:x+w]
                cv2.imwrite(os.path.join(debug_folder, "license_plate.jpg"), plate_debug)
            break

    if license_plate is None:
        return "Tablica nie znaleziona"

    # Wyostrzanie tablicy
    license_plate = cv2.GaussianBlur(license_plate, (3, 3), 0)
    license_plate = cv2.threshold(license_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "license_plate_processed.jpg"), license_plate)

    # Segmentacja znaków
    contours, _ = cv2.findContours(license_plate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    characters_folder = os.path.join(debug_folder, "characters")
    if debug:
        os.makedirs(characters_folder, exist_ok=True)

    characters = []
    for contour in sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0]):
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10:  # Filtracja małych konturów (szumów)
            char = license_plate[y:y+h, x:x+w]
            char = cv2.resize(char, (30, 50))  # Normalizacja rozmiaru znaków
            characters.append(char)
            if debug:
                char_debug_path = os.path.join(characters_folder, f"char_{x}.jpg")
                cv2.imwrite(char_debug_path, char)

    # Użycie pytesseract do odczytu tekstu (jak model wytrenujemy, wstawimy go w tym momencie!)
    config = "--psm 7"  # Tryb segmentacji: pojedyncza linia tekstu
    text = pytesseract.image_to_string(license_plate, config=config)
    return text.strip()

def main():
    dataset_path = '.'
    output_file = 'results.txt'

    if not os.path.exists(dataset_path):
        print(f"Folder {dataset_path} nie istnieje!")
        return

    results = []
    for file_name in os.listdir(dataset_path):
        if file_name.endswith('.jpg'):
            image_path = os.path.join(dataset_path, file_name)
            print(f"Przetwarzanie: {image_path}")
            text = process_image(image_path, debug=True)
            results.append(f"{file_name}: {text}")

    with open(output_file, 'w') as f:
        for result in results:
            f.write(result + '\n')

    print(f"Wyniki zapisano w pliku {output_file}")

if __name__ == "__main__":
    main()

Przetwarzanie: ./polen34.jpg
Przetwarzanie: ./polen33.jpg
Przetwarzanie: ./polen35.jpg
Przetwarzanie: ./polen25.jpg
Przetwarzanie: ./polen17.jpg
Wyniki zapisano w pliku results.txt


**Wersja 2 - Usunięto zapisywanie do pliku tekstowego, zamiast tego zwracamy tablicę characters przechowującą pojedyncze znaki wyciągnięte z tablicy rejestracyjnej.**

In [14]:
import cv2
import pytesseract
import os

def process_image(image_path, debug=False):
    # Tworzenie folderu debug
    debug_folder = "debug"
    if debug:
        os.makedirs(debug_folder, exist_ok=True)

    # Wczytanie obrazu
    image = cv2.imread(image_path)

    # Konwersja do skali szarości
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "gray.jpg"), gray)

    # Redukcja szumów
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "blurred.jpg"), blurred)

    # Adaptacyjne progowanie
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "thresh.jpg"), thresh)

    # Znalezienie konturów
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    license_plate = None
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h
        if 2 < aspect_ratio < 5 and h > 20:
            license_plate = gray[y:y+h, x:x+w]
            if debug:
                plate_debug = image[y:y+h, x:x+w]
                cv2.imwrite(os.path.join(debug_folder, "license_plate.jpg"), plate_debug)
            break

    if license_plate is None:
        return "Tablica nie znaleziona"

    # Wyostrzanie tablicy
    license_plate = cv2.GaussianBlur(license_plate, (3, 3), 0)
    license_plate = cv2.threshold(license_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    if debug:
        cv2.imwrite(os.path.join(debug_folder, "license_plate_processed.jpg"), license_plate)

    # Segmentacja znaków
    contours, _ = cv2.findContours(license_plate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    characters_folder = os.path.join(debug_folder, "characters")
    if debug:
        os.makedirs(characters_folder, exist_ok=True)

    characters = []
    for contour in sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0]):
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10:  # Filtracja małych konturów (szumów)
            char = license_plate[y:y+h, x:x+w]
            char = cv2.resize(char, (30, 50))  # Normalizacja rozmiaru znaków
            characters.append(char)
            if debug:
                char_debug_path = os.path.join(characters_folder, f"char_{x}.jpg")
                cv2.imwrite(char_debug_path, char)

    # Użycie pytesseract do odczytu tekstu (jak model wytrenujemy, wstawimy go w tym momencie!)
    config = "--psm 7"  # Tryb segmentacji: pojedyncza linia tekstu
    text = pytesseract.image_to_string(license_plate, config=config)
    return text.strip()

def main():
    dataset_path = '.'

    if not os.path.exists(dataset_path):
        print(f"Folder {dataset_path} nie istnieje!")
        return

    all_characters = []

    for file_name in os.listdir(dataset_path):
        if file_name.endswith('.jpg'):
            image_path = os.path.join(dataset_path, file_name)
            print(f"Przetwarzanie: {image_path}")
            text = process_image(image_path, debug=True)

            if text != "Tablica nie znaleziona":
                characters = list(text.replace(" ", ""))
                all_characters.append(characters)
            else:
                all_characters.append(["Brak tablicy"])

    return all_characters

if __name__ == "__main__":
    characters_table = main()
    for row in characters_table:
        print(row)


Przetwarzanie: ./polen34.jpg
Przetwarzanie: ./polen35.jpg
Przetwarzanie: ./polen25.jpg
Przetwarzanie: ./polen17.jpg
['A', 'B', 'L', '1', '9', '9', '5', '7']
['B', 'B', 'K', 'L', '0', '3', '6', '9', '3']
['G', 'W', 'E', '9', '1', '7', '1']
['G', 'A', '0', '9', '1', '7', 'C', 'i', 'm']
